# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
# Libraries
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
step1 = data.groupby(['CustomerID','ProductName']).sum('Quantity')[['Quantity']]
step1

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pivot = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum', fill_value=0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
tabla_final = pivot.T
tabla_final

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


In [8]:
dist_calculation = pdist(X=tabla_final, metric='euclidean')
dist_calculation

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [9]:
dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=tabla_final.index, 
                           columns=tabla_final.index)

euclid_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [11]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(tabla_final, 'euclidean'))),
                                index=tabla_final.index,
                                columns=tabla_final.index)

euclid_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [12]:
euclid_simil_norm = euclid_dist_norm[98200].sort_values(ascending=False)[1:]
type(euclid_simil_norm.head())

pandas.core.series.Series

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
top5 = euclid_simil_norm.head().index.tolist()
top5

[11883, 14782, 32324, 14208, 32180]

In [14]:
registros_seleccionados = step1.loc[top5]
registros_seleccionados

Quantity
CustomerID ProductName                                
11883      Artichokes - Jerusalem                    4
           Assorted Desserts                         4
           Bandage - Fexible 1x3                     4
           Bandage - Flexible Neon                   4
           Bar Mix - Pina Colada, 355 Ml             4
...                                                ...
32180      Wine - Cahors Ac 2000, Clos               9
           Wine - Chardonnay South                   9
           Wine - Fume Blanc Fetzer                  9
           Wine - Redchard Merritt                   9
           Wine - Vineland Estate Semi - Dry         9

[345 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [15]:
df_quant_top5 = registros_seleccionados.reset_index().groupby(['ProductName']).sum('Quantity').sort_values(by='Quantity', ascending= False)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [16]:
df_merged = pd.merge(df_quant_top5, pivot, left_index=True, right_index=True)
df_merged

,CustomerID,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Sea Bass - Whole,78712,30,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,25,50
Beans - Wax,64504,27,0,0,0,0,1,0,0,0,...,25,0,0,0,0,0,25,0,25,0
Apricots - Halves,64504,27,0,0,1,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
Sherry - Dry,58989,26,0,1,0,0,0,0,0,0,...,25,0,25,0,0,25,0,0,0,0
Pork - Hock And Feet Attached,44063,22,1,0,1,0,0,0,0,1,...,0,25,0,0,0,25,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
General Purpose Trigger,11883,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Guinea Fowl,14208,4,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
Halibut - Steaks,11883,4,0,1,0,0,0,0,0,1,...,0,0,0,25,0,0,0,0,25,0


In [17]:
df_merged1 = df_merged[df_merged[98200] == 0]
df_merged1

,CustomerID,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Beans - Wax,64504,27,0,0,0,0,1,0,0,0,...,25,0,0,0,0,0,25,0,25,0
Sherry - Dry,58989,26,0,1,0,0,0,0,0,0,...,25,0,25,0,0,25,0,0,0,0
Pork - Hock And Feet Attached,44063,22,1,0,1,0,0,0,0,1,...,0,25,0,0,0,25,0,0,0,0
Pears - Bosc,46388,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,25,0,0,0
"Soup - Campbells, Cream Of",78712,22,0,0,0,0,0,1,1,0,...,25,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Garbag Bags - Black,11883,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
General Purpose Trigger,11883,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Guinea Fowl,14208,4,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0


In [18]:
df_merged1.reset_index()[['ProductName','Quantity']].head()

,ProductName,Quantity
0,Beans - Wax,27
1,Sherry - Dry,26
2,Pork - Hock And Feet Attached,22
3,Pears - Bosc,22
4,"Soup - Campbells, Cream Of",22


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [19]:
recom = {}
unique = data['CustomerID'].unique().tolist()

In [20]:
for i in unique:
    euclid_simil_norm = euclid_dist_norm[i].sort_values(ascending=False)[1:]
    selec = step1.loc[euclid_simil_norm.head().index.tolist()]
    df_quant_top5 = selec.reset_index().groupby(['ProductName']).sum('Quantity').sort_values(by='Quantity', ascending= False)
    df_merged = pd.merge(df_quant_top5, pivot, left_index=True, right_index=True)
    df_merged1 = df_merged[df_merged[i] == 0]
    df_merged_final = df_merged1.reset_index()[['ProductName','Quantity']].head()
    recom[i] = df_merged_final['ProductName']

In [21]:
recom

{61288: 0                 Jagermeister
 1          Chicken - Soup Base
 2         Lime Cordial - Roses
 3    Macaroons - Two Bite Choc
 4          Flavouring - Orange
 Name: ProductName, dtype: object,
 77352: 0    Wine - Valpolicella Masi
 1                Tahini Paste
 2         Tuna - Salad Premix
 3         Chicken - Soup Base
 4                     Isomalt
 Name: ProductName, dtype: object,
 40094: 0    Bread - Italian Corn Meal Poly
 1                     Puree - Mocha
 2               Tuna - Salad Premix
 3                      Sherry - Dry
 4         Beer - Sleemans Cream Ale
 Name: ProductName, dtype: object,
 23548: 0    Squid - Tubes / Tenticles 10/20
 1        Sprouts - Baby Pea Tendrils
 2                Flavouring - Orange
 3         Bread - Raisin Walnut Oval
 4                     Banana Turning
 Name: ProductName, dtype: object,
 78981: 0                 Vanilla Beans
 1              Lettuce - Frisee
 2    Yogurt - Blueberry, 175 Gr
 3          Cinnamon Buns Sticky
 4 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [22]:
step9 = pd.DataFrame(recom)
step9

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Jagermeister,Wine - Valpolicella Masi,Bread - Italian Corn Meal Poly,Squid - Tubes / Tenticles 10/20,Vanilla Beans,Cocoa Butter,Juice - Lime,Bread - French Baquette,Soup Knorr Chili With Beans,Wine - White Cab Sauv.on,...,Beef - Top Sirloin,"Pasta - Penne, Rigate, Dry",Milk - 2%,Cheese - Mix,Chips Potato Salt Vinegar 43g,"Pasta - Penne, Rigate, Dry",Veal - Inside,Olives - Kalamata,"Veal - Brisket, Provimi,bnls","Pepper - Black, Whole"
1,Chicken - Soup Base,Tahini Paste,Puree - Mocha,Sprouts - Baby Pea Tendrils,Lettuce - Frisee,"Cheese - Boursin, Garlic / Herbs",Tomatoes Tear Drop,Bandage - Flexible Neon,Soupfoamcont12oz 112con,Cheese - Mozzarella,...,Grouper - Fresh,Coffee - Hazelnut Cream,Bay Leaf,Rum - Mount Gay Eclipes,Fenngreek Seed,Browning Caramel Glace,"Thyme - Lemon, Fresh",Wine - White Cab Sauv.on,Beef - Ground Medium,Knife Plastic - White
2,Lime Cordial - Roses,Tuna - Salad Premix,Tuna - Salad Premix,Flavouring - Orange,"Yogurt - Blueberry, 175 Gr",Garlic - Peeled,Pomello,"Lentils - Red, Dry",Veal - Sweetbread,"Lemonade - Natural, 591 Ml",...,Soup Knorr Chili With Beans,Soup Knorr Chili With Beans,"Veal - Inside, Choice",Muffin Mix - Blueberry,Isomalt,Tea - Decaf Lipton,Wonton Wrappers,Veal - Eye Of Round,"Rosemary - Primerba, Paste",Butter - Unsalted
3,Macaroons - Two Bite Choc,Chicken - Soup Base,Sherry - Dry,Bread - Raisin Walnut Oval,Cinnamon Buns Sticky,Cheese - Mix,"Potatoes - Instant, Mashed",Thermometer Digital,Cocktail Napkin Blue,Appetizer - Sausage Rolls,...,Eggplant - Asian,Sardines,Banana Turning,Cake - Box Window 10x10x2.5,Bay Leaf,"Garlic - Primerba, Paste","Veal - Inside, Choice",Bread - Calabrese Baguette,Scallops - 10/20,Wine - Ej Gallo Sierra Valley
4,Flavouring - Orange,Isomalt,Beer - Sleemans Cream Ale,Banana Turning,Pop Shoppe Cream Soda,Sauce - Hollandaise,Table Cloth - 53x69 Colour,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Quiche Assorted,...,Lime Cordial - Roses,"Chicken - Leg, Boneless",Wine - Vidal Icewine Magnotta,Scallops 60/80 Iqf,Veal - Sweetbread,"Wine - Red, Harrow Estates, Cab",Bread Crumbs - Japanese Style,Cheese - Wine,"Oregano - Dry, Rubbed",Wine - Blue Nun Qualitatswein


In [23]:
step9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Columns: 1000 entries, 61288 to 3472
dtypes: object(1000)
memory usage: 39.2+ KB


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [24]:
ham_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(tabla_final, 'hamming'))),
                                index=tabla_final.index,
                                columns=tabla_final.index)
ham_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.795775,0.826325,0.820327,0.801418,0.811490,0.795775,0.808587,0.805704,0.811490,...,0.788831,0.797178,0.797178,0.804270,0.794376,0.792982,0.791594,0.794376,0.776632,0.792982
200,0.795775,1.000000,0.797178,0.794376,0.771331,0.787456,0.792982,0.790210,0.797178,0.801418,...,0.779310,0.784722,0.787456,0.795775,0.779310,0.779310,0.787456,0.783362,0.762226,0.773973
264,0.826325,0.797178,1.000000,0.807143,0.811490,0.804270,0.794376,0.801418,0.804270,0.802842,...,0.791594,0.795775,0.792982,0.807143,0.795775,0.802842,0.794376,0.797178,0.782007,0.788831
356,0.820327,0.794376,0.807143,1.000000,0.800000,0.801418,0.794376,0.818841,0.800000,0.805704,...,0.788831,0.795775,0.795775,0.807143,0.791594,0.795775,0.784722,0.798587,0.782007,0.791594
412,0.801418,0.771331,0.811490,0.800000,1.000000,0.795775,0.794376,0.810036,0.792982,0.802842,...,0.776632,0.794376,0.784722,0.798587,0.784722,0.788831,0.777969,0.787456,0.775300,0.790210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.792982,0.779310,0.802842,0.795775,0.788831,0.797178,0.788831,0.801418,0.795775,0.795775,...,0.798587,0.815884,0.800000,0.821818,0.810036,1.000000,0.811490,0.814414,0.782007,0.784722
98069,0.791594,0.787456,0.794376,0.784722,0.777969,0.786087,0.780656,0.792982,0.790210,0.792982,...,0.790210,0.798587,0.795775,0.802842,0.808587,0.811490,1.000000,0.811490,0.784722,0.801418
98159,0.794376,0.783362,0.797178,0.798587,0.787456,0.795775,0.787456,0.798587,0.798587,0.797178,...,0.794376,0.817360,0.807143,0.811490,0.801418,0.814414,0.811490,1.000000,0.797178,0.795775


In [25]:
recom = {}
unique = data['CustomerID'].unique().tolist()

In [26]:
for i in unique:
    ham_simil_norm = ham_dist_norm[i].sort_values(ascending=False)[1:]
    selec = step1.loc[ham_simil_norm.head().index.tolist()]
    df_quant_top5 = selec.reset_index().groupby(['ProductName']).sum('Quantity').sort_values(by='Quantity', ascending= False)
    df_merged = pd.merge(df_quant_top5, pivot, left_index=True, right_index=True)
    df_merged1 = df_merged[df_merged[i] == 0]
    df_merged_final = df_merged1.reset_index()[['ProductName','Quantity']].head()
    recom[i] = df_merged_final['ProductName']

In [27]:
step10 = pd.DataFrame(recom)
step10

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,"Mushrooms - Black, Dried",Bandage - Fexible 1x3,Cinnamon Buns Sticky,Beef - Rib Eye Aaa,Duck - Breast,Kiwi,Wine - Ruffino Chianti,Pastry - Cheese Baked Scones,"Lamb - Pieces, Diced","Sauce - Gravy, Au Jus, Mix",...,Ecolab - Lime - A - Way 4/4 L,"Wine - Red, Cooking",Cumin - Whole,Mussels - Frozen,Beef Wellington,"Wine - Cahors Ac 2000, Clos",Pastry - Cheese Baked Scones,"Liners - Banana, Paper","Wine - Cahors Ac 2000, Clos","Appetizer - Mini Egg Roll, Shrimp"
1,Wine - Chablis 2003 Champs,Nantuket Peach Orange,Sponge Cake Mix - Chocolate,Napkin White - Starched,Muffin - Zero Transfat,Ice Cream Bar - Drumstick,"Lamb - Pieces, Diced",Beef Wellington,Wine - Ruffino Chianti,Beef - Rib Eye Aaa,...,Napkin White - Starched,Pastry - Cheese Baked Scones,Cookie Dough - Double,"Mushrooms - Black, Dried","Mushrooms - Black, Dried","Wine - White, Mosel Gold",Beef Wellington,"Fish - Scallops, Cold Smoked","Mushrooms - Black, Dried",Beef Wellington
2,"Oil - Shortening,liqud, Fry",Beef Ground Medium,Wine - Chardonnay South,"Appetizer - Mini Egg Roll, Shrimp",Cookie Dough - Double,Duck - Breast,Sunflower Seed Raw,Oil - Shortening - All - Purpose,Sunflower Seed Raw,Beer - Original Organic Lager,...,Sage - Ground,Beef Wellington,Duck - Breast,Pastry - Cheese Baked Scones,Rambutan,Garbag Bags - Black,"Doilies - 5, Paper",Fuji Apples,Cumin - Whole,"Sauce - Gravy, Au Jus, Mix"
3,"Lemonade - Natural, 591 Ml",Flour - Whole Wheat,Browning Caramel Glace,"Lamb - Pieces, Diced","Lamb - Pieces, Diced",Pastry - Cheese Baked Scones,Beer - Original Organic Lager,Cumin - Whole,Beer - Original Organic Lager,Cocktail Napkin Blue,...,Cumin - Whole,"Doilies - 5, Paper",Wine - Ej Gallo Sierra Valley,"Wine - Cahors Ac 2000, Clos","Liners - Banana, Paper",Mussels - Frozen,"Wine - Red, Cooking","Mushrooms - Black, Dried",Ecolab - Lime - A - Way 4/4 L,Napkin White - Starched
4,"Wine - Cahors Ac 2000, Clos","Ice - Clear, 300 Lb For Carving","Juice - Cranberry, 341 Ml",Sage - Ground,French Pastry - Mini Chocolate,"Placemat - Scallop, White",Pail For Lid 1537,"Doilies - 5, Paper",Beef Wellington,Bandage - Fexible 1x3,...,"Sauce - Gravy, Au Jus, Mix",Table Cloth 62x114 White,Puree - Mocha,Kiwi,Sunflower Seed Raw,"Lamb - Pieces, Diced","Turkey - Whole, Fresh",Veal - Inside,"Juice - Cranberry, 341 Ml",Banana - Leaves
